In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import os
import re
import json
import tqdm
import pickle
import openai
import pathlib
import pandas as pd

from typing import List
from openai import embeddings_utils as utils

In [22]:
ORGANIZATION = ""
API_KEY = ""

In [23]:
openai.organization = ORGANIZATION
openai.api_key = API_KEY

## Embeddings

### Build cache

In [24]:
EMBEDDING_CACHE_PATH = "data/embedding_cache.pickle"

In [25]:
try:
    EMBEDDING_CACHE = pd.read_pickle(EMBEDDING_CACHE_PATH)
except FileNotFoundError:
    EMBEDDING_CACHE = {}

In [26]:
def save_cache():
    with open(EMBEDDING_CACHE_PATH, "wb") as embedding_cache_file:
        pickle.dump(EMBEDDING_CACHE, embedding_cache_file)

In [27]:
def embedding_from_string(string: str, engine: str = "text-similarity-babbage-001", save: bool = True) -> List:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, engine) not in EMBEDDING_CACHE.keys():
        EMBEDDING_CACHE[(string, engine)] = utils.get_embedding(string, engine)
        
        if save:
            save_cache()

    return EMBEDDING_CACHE[(string, engine)]

### Search

In [28]:
def find_best_documents(
    query: str,
    documents: List[dict],
    engine: str= "text-similarity-babbage-001",
    count: int = 1
) -> List[int]:
    query_embedding = embedding_from_string(query, engine=engine)
    embeddings = [embedding_from_string(doc["content"], engine=engine) for doc in documents]
    distances = utils.distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine")
    indices_of_nearest_neighbors = utils.indices_of_nearest_neighbors_from_distances(distances)

    results = []
    count = min(count, len(documents))
    for idx in indices_of_nearest_neighbors[:count]:
        document = documents[idx]
        results.append({
            "id": document["id"],
            "index": idx,
            "content": document["content"],
            "distance": distances[idx]
        })

    return results

### Precompute

In [92]:
df = pd.read_csv("data/data.csv")
df = df.fillna("")

In [179]:
MAPPING = {
    "mau": "monthly active users",
    "dau": "daily active user",
    "eth": "ethereum",
    "sol": "solana",
    "bnb": "binance",
    "lens": "lens protocol",
    "ens": "ethereum name service",
    "tvl": "total value locked",
    "avg": "average",
    "mean": "average",
    "tx": "transcation",
    "$": "dollars"
}

In [180]:
def normalize(text: str) -> str:
    words = re.split("[,.!?:() ]", text)
    words = [w.lower() for w in words if len(w) > 0]
    words = [MAPPING.get(w, w) for w in words]
    return " ".join(words)

In [187]:
def extract_desc(visualizations: str) -> str:
    result = ""
    visualizations = json.loads(visualizations)
    
    for v in visualizations:
        if len(v["name"].split()) <= 1:
            continue
        
        if v["type"] != "counter":
            continue

        desc = f"use {v['options']['counterColName']} for {v['name'].lower()}"
        desc = normalize(desc)
        
        if len(result) > 0:
            result += ", "
        
        result += desc

    return result

In [190]:
contents = []
for i, query in df.iterrows():
    content = f"{query['name']} {query['description']}".strip()
    content = normalize(content)

    desc = extract_desc(query["visualizations"])
    if len(desc) > 0:
        content += f" ({desc})"
    
    content = content.strip()
    contents.append(content)

In [193]:
df["content"] = contents

In [195]:
df.to_csv("data/data.csv", index=False)

In [196]:
documents = []
for i, query in df.iterrows():
    documents.append({"id": query.id, "content": query.content})

In [198]:
EMBEDDING_CACHE = {}

In [199]:
for document in tqdm.tqdm(documents):
    embedding_from_string(document["content"], engine="text-similarity-babbage-001", save=False)

100%|██████████| 3638/3638 [07:01<00:00,  8.64it/s]


In [200]:
save_cache()

In [206]:
with open("prompt.txt", "w") as f:
    f.write("WITH all_trades AS\n(SELECT\nt.block_time,\nCASE --filling in missing addresses for coins. Maybe not worth keeping up, some are memes.\nWHEN t.token_a_address::text = '\\x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce' THEN 'SHIB'\nWHEN t.token_a_address::text = '\\x761d38e5ddf6ccf6cf7c55759d5210750b5d60f3' THEN 'ELON'\nWHEN t.token_a_address::text = '\\x3301ee63fb29f863f2333bd4466acb46cd8323e6' THEN 'AKITA'\nWHEN t.token_a_address::text = 'x5218e472cfcfe0b64a064f055b43b4cdc9efd3a6' THEN 'eRSDL'\nWHEN t.token_a_address::text = '\\x1453dbb8a29551ade11d89825ca812e05317eaeb' THEN 'TEND'\nWHEN t.token_a_address::text = '\\x35a532d376ffd9a705d0bb319532837337a398e7' THEN 'WDOGE'\nWHEN t.token_a_address::text = '\\x17ef75aa22dd5f6c2763b8304ab24f40ee54d48a' THEN 'RVP'\nWHEN t.token_a_address::text = '\\x6dea81c8171d0ba574754ef6f8b412f2ed88c54d' THEN 'LQTY'\nWHEN t.token_a_address::text = '\\x3832d2f059e55934220881f831be501d180671a7' THEN 'renDOGE'\nWHEN t.token_a_address::text = '\\xf16e4d813f4dcfde4c5b44f305c908742de84ef0' THEN 'ETH2x ADL'\nWHEN t.token_a_address::text = '\\x77fba179c79de5b7653f68b5039af940ada60ce0' THEN 'FORTH'\nWHEN t.token_a_address::text = '\\xf65b5c5104c4fafd4b709d9d60a185eae063276c' THEN 'TRU'\nWHEN t.token_a_address::text = '\\xc7283b66eb1eb5fb86327f08e1b5816b0720212b' THEN 'TRIBE'\nWHEN t.token_a_address::text = '\\xc8807f0f5ba3fa45ffbdc66928d71c5289249014' THEN 'ISP'\nELSE COALESCE(t.token_a_symbol,t.token_a_address::text)\nEND AS token_a,\nCASE --filling in missing addresses for coins. Maybe not worth keeping up, some are memes.\nWHEN t.token_b_address::text = '\\x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce' THEN 'SHIB'\nWHEN t.token_b_address::text = '\\x761d38e5ddf6ccf6cf7c55759d5210750b5d60f3' THEN 'ELON'\nWHEN t.token_b_address::text = '\\x3301ee63fb29f863f2333bd4466acb46cd8323e6' THEN 'AKITA'\nWHEN t.token_b_address::text = 'x5218e472cfcfe0b64a064f055b43b4cdc9efd3a6' THEN 'eRSDL'\nWHEN t.token_b_address::text = '\\x1453dbb8a29551ade11d89825ca812e05317eaeb' THEN 'TEND'\nWHEN t.token_b_address::text = '\\x35a532d376ffd9a705d0bb319532837337a398e7' THEN 'WDOGE'\nWHEN t.token_b_address::text = '\\x17ef75aa22dd5f6c2763b8304ab24f40ee54d48a' THEN 'RVP'\nWHEN t.token_b_address::text = '\\x6dea81c8171d0ba574754ef6f8b412f2ed88c54d' THEN 'LQTY'\nWHEN t.token_b_address::text = '\\x3832d2f059e55934220881f831be501d180671a7' THEN 'renDOGE'\nWHEN t.token_b_address::text = '\\xf16e4d813f4dcfde4c5b44f305c908742de84ef0' THEN 'ETH2x ADL'\nWHEN t.token_b_address::text = '\\x77fba179c79de5b7653f68b5039af940ada60ce0' THEN 'FORTH'\nWHEN t.token_b_address::text = '\\xf65b5c5104c4fafd4b709d9d60a185eae063276c' THEN 'TRU'\nWHEN t.token_b_address::text = '\\xc7283b66eb1eb5fb86327f08e1b5816b0720212b' THEN 'TRIBE'\nWHEN t.token_b_address::text = '\\xc8807f0f5ba3fa45ffbdc66928d71c5289249014' THEN 'ISP'\nELSE COALESCE(t.token_b_symbol,t.token_b_address::text)\nEND AS token_b\n,t.token_a_amount\n,t.token_b_amount\n,t.exchange_contract_address -- exchange_contract_address is the v3 LP Position\n,t.tx_hash -- transaction addr (for spot checks)\n,t.usd_amount --TODO: Find some way to pull in USD of any missing tokens\n,c.fee/1e6 AS fee_tier --Converting fee tier to a multiplier (i.e. for 0.3%, 3000 -> 0.003)\n,t.usd_amount*(c.fee/1e6) AS fees_collected_usd --USD Amount * %fee = fees collected\nFROM dex.\"trades\" t\n\nLEFT JOIN uniswap_v3.\"Factory_call_createPool\" c --1. Join With Pool creation on LP contract address to pull fee tier\nON t.exchange_contract_address = c.output_pool\nWHERE t.project = 'Uniswap'\nAND t.version = '3'\nAND t.block_time >= (DATE_TRUNC('day',CURRENT_TIMESTAMP) - '6 days'::INTERVAL)\nAND t.block_time >= '05-05-2021 17:00' -- Uni v3 launch date/hr\nORDER BY t.block_time DESC\n),\n\nsum_pairs AS( -- Using this to decide which trading pair order to pick (i.e WBTC/USDC vs USDC/WBTC). We'll pick the one with the greatest usd volume.\n--starting set\nSELECT \nCONCAT(token_a,'/',token_b) AS forward_pair -- Trading pair\n,CONCAT(token_b,'/',token_a) AS backward_pair -- For joining buys/sells later\n,SUM(usd_amount) AS sum_usd\nFROM all_trades\nGROUP BY 1,2\n),\n\nfinal_trading_pairs AS( --building the mapping table for pairs\nSELECT\na.forward_pair, a.backward_pair,\na.sum_usd,b.sum_usd,\nCASE\nWHEN RIGHT(a.backward_pair,4)='WETH' THEN a.forward_pair -- if the backward version ends in WETH, do forward\nWHEN RIGHT(a.forward_pair,4)='WETH' THEN a.backward_pair\nWHEN RIGHT(a.backward_pair,4)='WBTC' THEN a.forward_pair\nWHEN RIGHT(a.forward_pair,4)='WBTC' THEN a.backward_pair\nWHEN (a.sum_usd >= b.sum_usd) -- when a > b\nOR b.sum_usd IS NULL -- or if backward usd is null\nTHEN a.forward_pair -- select forward\nELSE a.backward_pair --else select backward in every other circumstance\nEND AS trading_pair\n\nFROM sum_pairs a\nLEFT JOIN sum_pairs b\nON a.forward_pair = b.backward_pair\n),\nv3_trades AS (  --clean table of trades to work with for building visuals\nSELECT\natx.block_time,\nCOALESCE(f1.trading_pair,f2.trading_pair) AS trading_pair, --pick the mapped trading pair (see joins)\natx.usd_amount,\natx.fee_tier,\natx.fees_collected_usd,\natx.tx_hash\nFROM all_trades atx\n\nLEFT JOIN final_trading_pairs f1 --check if the pair maps with the forward version\nON CONCAT(atx.token_a,'/',atx.token_b) = f1.forward_pair\nLEFT JOIN final_trading_pairs f2 --check if the pair maps with the backward version\nON CONCAT(atx.token_a,'/',atx.token_b) = f2.backward_pair\n)\n\nSELECT\ntrading_pair\n,COUNT(trading_pair) AS num_trades\n,SUM(usd_amount) AS total_usd\nFROM v3_trades\nWHERE usd_amount IS NOT NULL -- Eventually see if Dune adds new tokens, or we need to manually\nGROUP BY trading_pair--, fee_tier\nORDER BY SUM(usd_amount) DESC")

## Generate SQL

In [14]:
df = pd.read_csv("data/data.csv")

In [15]:
documents = []
for i, query in df.iterrows():
    documents.append({
        "id": query.id,
        "content": f"{query['name']} {query.description or ''}".strip()
    })

In [121]:
query = "number of ens registered accounts"
recommendations = find_best_documents(query, documents, count=20)
recommendations = [r for r in recommendations if r["distance"] < 0.25]

In [122]:
len(recommendations)

20

In [123]:
prompt = ""
for doc in recommendations:
    raw_doc = df.iloc[doc["index"]]
    
    prompt += f"/* Write SQL query: {raw_doc['name']} */\n"
    prompt += raw_doc["query"].strip()
    prompt += "\n\n"

    if len(prompt) >= 6000:
        break

prompt += f"/* Write SQL query: {query} */\n"

In [126]:
result = openai.Completion.create(
  model="code-davinci-002",
  prompt=prompt,
  max_tokens=512,
  temperature=0,
  stop=["STOP", "/* Write SQL"]
)

In [132]:
completion = result.choices[0].text.strip()
print(completion)

SELECT COUNT(*) FROM (
    SELECT DISTINCT(owner) FROM (
        SELECT * FROM ethereumnameservice."ENSRegistry_evt_NewOwner"
        UNION
        SELECT * FROM ethereumnameservice."ENSRegistryWithFallback_evt_NewOwner"
    ) as rr
) as r
